# Realigning every listening subject

There is, for every listen subject, both a stretch, and an offset that need to be considered.

Let's build a dataframe with the information, for each run, of the offset found and the stretch.

Finally, plot the un-stretched and with offset data, compared to the metadata.

## Generate audio ground truth to calculate stretch

In [ ]:
# from utils import gen_audio_words_length
import numpy as np

# audio_word_length = gen_audio_words_length()

# # Save this file to csv not to have to load it everytime

# # Save it to numpy file
# np.save('audio_word_length.npy', audio_word_length)


audio_word_length = np.load('audio_word_length.npy')



## Find the offset and stretch

In [ ]:
from utils import get_raw
import pandas as pd
from tqdm import tqdm

df = pd.DataFrame(columns=['subject', 'run', 'offset', 'stretch'])

for subject in tqdm(range(1,59), desc='Subjects'):
    for run in tqdm(range(1,10), desc='Runs', leave=False):
        try:
            raw, word_triggers, events = get_raw(str(subject), run)
            # Calculate the offset between the first word trigger and the first sample
            offset = (word_triggers[0] - raw.first_samp) / raw.info['sfreq']
            # And add the initial time between the first word trigger and the first sample (eg. 3.05 for run 1)
            offset -= events.onset[0]
            # Get the length of the recorded word triggers
            meg_word_length = (word_triggers[-1] - word_triggers[0]) / raw.info['sfreq']
            # Deduct the stretch factor
            stretch = meg_word_length / audio_word_length[run-1]

            # Append the offset and the stretch factor to the dataframe using concat
            df = pd.concat([df, pd.DataFrame({'subject': [subject], 'run': [run], 'offset': [offset], 'stretch': [stretch]})], ignore_index=True)
            # Save the dataframe
            df.to_csv('infos2.csv', index=False)
        except:
            print(f'Error with subject {subject} run {run}')
            continue

## Testing the alignment! 

In [ ]:
import mne
import logging
from utils import save_multiplot
# Set the logger level to WARNING to reduce verbosity
logger = logging.getLogger('mne')
logger.setLevel(logging.ERROR)

for subject in tqdm(range(1,2), desc='Subjects'):
    for run in tqdm(range(1,3), desc='Runs'):
        save_multiplot(subject, run)

In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')

raw.pick_types(stim=True, misc=True).plot(block=True, start=0, duration=10., precompute='auto')